In [54]:
import functools

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [55]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file(fname="train.csv", origin=TRAIN_DATA_URL, cache_dir="./")
test_file_path = tf.keras.utils.get_file(fname="eval.csv", origin=TEST_DATA_URL, cache_dir="./")

In [56]:
# 让 numpy 数据更易读。保留小数 3 位
np.set_printoptions(precision=3, suppress=True)

In [57]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [58]:
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12, # 为了示例更容易展示，手动设置较小的值
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [59]:
examples, labels = next(iter(raw_train_data)) # 第一个批次
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'male', b'female', b'male', b'male', b'female', b'male',
       b'female', b'male', b'male', b'female', b'male', b'female'],
      dtype=object)>), ('age', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([40. , 35. , 18. , 28. , 36. , 28. , 47. , 28. , 32.5, 24. , 33. ,
       26. ], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1], dtype=int32)>), ('parch', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([4, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1], dtype=int32)>), ('fare', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([27.9  , 20.25 , 73.5  ,  7.896, 26.   ,  0.   , 52.554,  7.225,
       30.071, 14.5  , 12.275, 26.   ], dtype=float32)>), ('class', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'Third', b'Third', b'Second', b'Third', b'Second', b'Second',
       b'First', b'Third', b'Second', b'Second', b'Se

## 数据处理 layer

In [60]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [61]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [62]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [63]:
def process_continuous_data(mean, data):
    # 标准化数据
    data = tf.cast(data, tf.float32) * 1/(2*mean)
    return tf.reshape(data, [-1, 1])

In [64]:
MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []

for feature in MEANS.keys():
    num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
    numerical_columns.append(num_col)

In [65]:
numerical_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7fd8f6708440>, 29.631308)),
 NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7fd8f6708440>, 0.545455)),
 NumericColumn(key='parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7fd8f6708440>, 0.379585)),
 NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7fd8f6708440>, 34.385399))]

## 创建预处理层
将这两个特征列的集合相加，并且传给 tf.keras.layers.DenseFeatures 从而创建一个进行预处理的输入层。

In [66]:
feature_concat = categorical_columns+numerical_columns
type(feature_concat)
print(feature_concat)

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, no

In [67]:
preprocessing_layer = tf.keras.layers.DenseFeatures(feature_concat)

In [68]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [71]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

In [72]:
history = model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 2ms/step - loss: 0.5291 - accuracy: 0.7560
Epoch 2/20
53/53 [==============================] - 0s 976us/step - loss: 0.4318 - accuracy: 0.8022
Epoch 3/20
53/53 [==============================] - 0s 1ms/step - loss: 0.4208 - accuracy: 0.8246
Epoch 4/20
53/53 [==============================] - 0s 1ms/step - loss: 0.4074 - accuracy: 0.8246
Epoch 5/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3942 - accuracy: 0.8373
Epoch 6/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3895 - accuracy: 0.8373
Epoch 7/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3895 - accuracy: 0.8182
Epoch 8/20
53/53 [==============================] - 0s 999us/step - loss: 0.3776 - accuracy: 0.8373
Epoch 9/20
53/53 [==============================] - 0s 1ms/step - loss: 0.3789 - accuracy: 0.8341
Epoc

In [73]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_3 (DenseFeatu multiple                  0         
_________________________________________________________________
dense_15 (Dense)             multiple                  3200      
_________________________________________________________________
dense_16 (Dense)             multiple                  16512     
_________________________________________________________________
dense_17 (Dense)             multiple                  129       
Total params: 19,841
Trainable params: 19,841
Non-trainable params: 0
_________________________________________________________________


In [78]:
history.history.items()

dict_items([('loss', [0.5291159749031067, 0.43181875348091125, 0.4208433926105499, 0.4073566794395447, 0.394185870885849, 0.38951316475868225, 0.3894624710083008, 0.3776199221611023, 0.3788665235042572, 0.36917349696159363, 0.3660622239112854, 0.3625626564025879, 0.35769304633140564, 0.36377131938934326, 0.35913950204849243, 0.3487794101238251, 0.3496316075325012, 0.3466752767562866, 0.34534794092178345, 0.3383704423904419]), ('accuracy', [0.7559808492660522, 0.8022328615188599, 0.8245614171028137, 0.8245614171028137, 0.8373205661773682, 0.8373205661773682, 0.8181818127632141, 0.8373205661773682, 0.8341307640075684, 0.8516746163368225, 0.840510368347168, 0.8341307640075684, 0.8452950716018677, 0.8484848737716675, 0.8484848737716675, 0.8468899726867676, 0.8484848737716675, 0.8500797152519226, 0.8564593195915222, 0.8612440228462219])])

In [74]:
test_loss, test_accuracy = model.evaluate(test_data)

Consider rewriting this model with the Functional API.
22/22 [==============================] - 0s 2ms/step - loss: 0.4575 - accuracy: 0.8030


In [75]:
predictions = model.predict(test_data)

# 显示部分结果
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Consider rewriting this model with the Functional API.
Predicted survival: 9.32%  | Actual outcome:  DIED
Predicted survival: 93.16%  | Actual outcome:  DIED
Predicted survival: 63.96%  | Actual outcome:  DIED
Predicted survival: 46.79%  | Actual outcome:  DIED
Predicted survival: 89.88%  | Actual outcome:  DIED
Predicted survival: 86.44%  | Actual outcome:  SURVIVED
Predicted survival: 12.88%  | Actual outcome:  DIED
Predicted survival: 82.93%  | Actual outcome:  DIED
Predicted survival: 29.37%  | Actual outcome:  SURVIVED
Predicted survival: 2.58%  | Actual outcome:  SURVIVED
